In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
df1 = pd.read_csv('dataset/7282_1.csv')
df2 = pd.read_csv('dataset/Datafiniti_Hotel_Reviews_Jun19.csv')
df3 = pd.read_csv('dataset/Datafiniti_Hotel_Reviews.csv')
df = pd.concat([df1,df2,df3],ignore_index=True)

In [3]:
df.dropna(axis = 'columns', how = 'all', inplace = True)
old_names = ['reviews.date', 'reviews.rating', 'reviews.title', 'reviews.text', 'reviews.userCity', 
             'reviews.username', 'reviews.userProvince'] 
new_names = ['date', 'rating', 'title', 'text', 'userCity', 'username', 'userProvince']
df.rename(columns = dict(zip(old_names, new_names)), inplace = True)
df.dropna(subset = ['rating', 'text'], inplace = True)
df['title'].fillna('', inplace = True)

<h1>Emotional Content of Reviews</h1>

In [4]:
import nltk
from nltk.corpus import PlaintextCorpusReader

In [5]:
def get_nrc_data():
    nrc = "data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
    count=0
    emotion_dict=dict()
    with open(nrc,'r') as f:
        all_lines = list()
        for line in f:
            if count < 46:
                count+=1
                continue
            # first 46 lines are descriptions
            line = line.strip().split('\t')
            if int(line[2]) == 1:
                #if there is no such key in the dictionary, create one list; if emotion_dict.get(line[0]) not None, append
                if emotion_dict.get(line[0]):
                    emotion_dict[line[0]].append(line[1])
                else:
                    emotion_dict[line[0]] = [line[1]]
    return emotion_dict

In [6]:
emotion_dict = get_nrc_data()
def emotion_analyzer(text,emotion_dict=emotion_dict):
    #Set up the result dictionary
    emotions = {x for y in emotion_dict.values() for x in y}
    emotion_count = dict()
    for emotion in emotions:
        emotion_count[emotion] = 0

    #Analyze the text and normalize by total number of words
    total_words = len(text.split())
    for word in text.split():
        if emotion_dict.get(word):
            for emotion in emotion_dict.get(word):
                emotion_count[emotion] += 1/len(text.split())
    return emotion_count

In [7]:
n_df = df.copy()
nrc_results = n_df['text'].apply(emotion_analyzer).apply(pd.Series)
df = pd.concat([df,nrc_results],axis=1)
df

,address,categories,city,country,latitude,longitude,name,postalCode,province,date,...,anticipation,trust,anger,negative,sadness,joy,surprise,fear,disgust,positive
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,...,0.030303,0.030303,0.030303,0.030303,0.000000,0.030303,0.030303,0.000,0.030303,0.090909
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,...,0.068182,0.090909,0.000000,0.000000,0.022727,0.068182,0.045455,0.000,0.000000,0.159091
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.035714
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,...,0.033898,0.050847,0.000000,0.000000,0.000000,0.050847,0.016949,0.000,0.000000,0.067797
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,...,0.033898,0.050847,0.000000,0.000000,0.000000,0.050847,0.016949,0.000,0.000000,0.067797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55907,215 S Pacific St,"Hotel,Hotels,Lodging,Motels",Rockaway Beach,US,45.611260,-123.945560,Silver Sands Oceanfront Motel,97136-9596,OR,2016-03-13T00:00:00Z,...,0.016393,0.032787,0.000000,0.000000,0.000000,0.032787,0.016393,0.000,0.000000,0.032787
55908,669 Route 6a,"Hotel,Hotels",East Sandwich,US,41.729390,-70.399250,Sandy Neck Motel,02537-1472,MA,2015-12-11T00:00:00Z,...,0.032787,0.016393,0.016393,0.016393,0.000000,0.032787,0.032787,0.000,0.016393,0.032787
55909,702 W Appleway Ave,"Hotel,Hotel, Motel, and Building,Hotels,Lodgin...",Coeur d'Alene,US,47.699930,-116.796900,Shilo Inn Suites - Coeur d'Alene,83814-9338,ID,2017-11-17T00:00:00.000Z,...,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000,0.000000,0.045455
55910,2295 N Highland Ave,"Hotel,Hotels Motels,Budget Hotels,Hotels & Motels",Jackson,US,35.666390,-88.830120,Scottish Inn,38305-4922,TN,2016-06-09T00:00:00Z,...,0.000000,0.000000,0.000000,0.016393,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
